In [0]:
import numpy as np
seed = 8
np.random.seed(seed)
import pickle
import pandas as pd
import tensorflow as tf
import math
from sklearn.preprocessing import LabelBinarizer
from random import shuffle

from tensorflow.python.client import device_lib
from tensorflow.keras import metrics 

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

get_available_devices()

In [0]:
def loadData():
  with open('C:\\Users\\Adjek\\Nextcloud\\DeeptechAI\\words2vec\\words2vec.pickle', 'rb') as f:
    data = pickle.load(f)
    return data
  
def createDataSet(enc, words, csv):
    
    X = list()
    Y = list()
    U = list()
    
    for url in words.keys():
      if(isinstance(csv[url], str) & (len(words[url]) == 20)):
        X.append(np.array(words[url]))
        Y.append(csv[url])
        U.append(url)
        
    Y = enc.fit_transform(Y)
    
    return np.array(X), np.array(Y), U

def reorderData(csv):
    urlList = csv["Domain"]
    sectionList = csv["WZ2008 Section"]
    
    resultDict = dict()
    for i in range(len(urlList)):
      resultDict[urlList[i]] = sectionList[i]
      
    return resultDict

In [0]:
data = pd.read_csv("C:\\Users\\Adjek\\Nextcloud\\DeeptechAI\\data\\urls.csv", sep = ";", encoding = "ISO-8859-1")
keyValueData = reorderData(data)
wordVectors = loadData()

encoder = LabelBinarizer()
dataSet_X, dataSet_Y, urls = createDataSet(encoder, wordVectors, keyValueData)

data.head()


In [0]:
print(dataSet_X[0])
print(len(dataSet_X))
print(dataSet_X.shape)
print(dataSet_Y[0])
print(len(dataSet_Y))
print(dataSet_Y.shape)
num_classes = dataSet_Y[1].shape[0]

In [0]:
#dataSet_X = dataSet_X.reshape(dataSet_X.shape[0], 20, 1)
#print(dataSet_X.shape)
#dataSet_Y = dataSet_Y.reshape(dataSet_Y.shape[0], 19, 1)
#print(dataSet_Y.shape)

In [0]:
trainSize = math.floor(len(dataSet_X) * 0.6)
validationSize = math.floor(len(dataSet_X) * 0.8)

#shuffle(dataSet_X)
#shuffle(dataSet_Y)


train_X = dataSet_X[0:trainSize]
train_Y = dataSet_Y[0:trainSize]
train_U = urls[0:trainSize]

validation_X = dataSet_X[trainSize:validationSize]
validation_Y = dataSet_Y[trainSize:validationSize]
validation_U = urls[trainSize:validationSize]

test_X = dataSet_X[validationSize:len(dataSet_X)]
test_Y = dataSet_Y[validationSize:len(dataSet_X)]
test_U = urls[validationSize:len(dataSet_X)]

print(len(train_X))
print(len(validation_X))
print(len(test_X))

In [0]:
print(train_X.shape)
print(train_X)
print(train_X[0].shape)

print(train_Y.shape)
print(train_Y)
print(train_Y[0].shape)

In [0]:
num_classes

In [0]:
def text_shallow_model():
    # create model
    inputs = tf.keras.Input(shape=(20, ))
    dense_1 = tf.keras.layers.Dense(450, activation=tf.nn.relu)(inputs)
    dense_2 = tf.keras.layers.Dense(30, activation=tf.nn.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)(dense_2)
    
    model = tf.keras.Model(inputs=inputs, outputs=dense_3)
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
    model.summary()
    return model
  
def text_deep_model():
    # create model
    inputs = tf.keras.Input(shape=(20, ))
    dense_1 = tf.keras.layers.Dense(250, activation=tf.nn.relu)(inputs)
    drop_1 = tf.keras.layers.Dropout(0.5)(dense_1)
    dense_2 = tf.keras.layers.Dense(100, activation=tf.nn.relu)(drop_1)
    drop_2 = tf.keras.layers.Dropout(0.5)(dense_2)
    dense_3 = tf.keras.layers.Dense(30, activation=tf.nn.relu)(drop_2)
    dense_4 = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)(dense_3)
    
    model = tf.keras.Model(inputs=inputs, outputs=dense_4)
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
    model.summary()
    return model

In [0]:
path = 'C:\\Jupyter'
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=path,
                                            write_graph=True,
                                            write_images=True,
                                            histogram_freq=0)

In [0]:
#@title Wähle das Device

# build the model
device = '/device:GPU:0' #@param ['/device:CPU:0', '/device:GPU:0']
with tf.device(device):
    model = text_shallow_model()
    
# Fit the model
    model.fit(train_X, train_Y, validation_data=(validation_X, validation_Y), epochs=200, batch_size=200)

# Final evaluation of the model
    scores = model.evaluate(test_X, test_Y, verbose=1)
print("Text Network Error: %.2f%%" % (100-scores[1]*100))

In [0]:
example_X = test_X[2:30]
example_Y = test_Y[2:30]
example_U = test_U[2:30]



prediction = model.predict(example_X)

predictedClasses = encoder.inverse_transform(prediction)
originalClasses = encoder.inverse_transform(example_Y)

In [0]:
#print(prediction)

for i in range(len(prediction)):
  print(str(prediction[i].argmax()) + "<->" + str(example_Y[i].argmax()) + " für URL: " + example_U[i] + " mit predicted Klasse " + predictedClasses[i] + " Lösung: " + originalClasses[i])